In [235]:
include("../src/JuliaMBD.jl")

Main.JuliaMBD

In [207]:
import .JuliaMBD

In [209]:
b = JuliaMBD.SimpleBlock(:test)
pout = JuliaMBD.OutPort(:a, Float64)
pin = JuliaMBD.InPort(:b, Float64)
JuliaMBD.set!(b, pout.name, pout)
JuliaMBD.set!(b, pin.name, pin)
pext = JuliaMBD.OutPort(:ex)
JuliaMBD.LineSignal(pext, pin, "s(t)")

"Line(##535)"

In [210]:
JuliaMBD.tsort(JuliaMBD.allcomponents(b))

4-element Vector{Main.JuliaMBD.AbstractComponent}:
 "OutPort(ex)"
 "InPort(b)"
 "Block(test)"
 "OutPort(a)"

In [211]:
bb = JuliaMBD.Gain()

"Block(Gain)"

In [216]:
b1 = JuliaMBD.Inport(:t, Float64)
b2 = JuliaMBD.Gain(in=JuliaMBD.InPort(:gainin), out=JuliaMBD.OutPort(:gainout))
b3 = JuliaMBD.Outport(:y, Float64)
JuliaMBD.ConstSignal(:K, b2.env[:K])
JuliaMBD.LineSignal(b1.outports[1], b2.env[:in], "s(t)")
JuliaMBD.LineSignal(b2.env[:out], b3.inports[1], "s(t)")

"Line(##548)"

In [217]:
JuliaMBD.tsort(JuliaMBD.allcomponents(b3))

10-element Vector{Main.JuliaMBD.AbstractComponent}:
 "InPort(t)"
 "Block(Inport)"
 "OutPort(##544)"
 "InPort(gainin)"
 "Parameter(##545)"
 "Block(Gain)"
 "OutPort(gainout)"
 "InPort(##546)"
 "Block(Outport)"
 "OutPort(y)"

In [218]:
expr = [JuliaMBD._expr(m) for m = JuliaMBD.tsort(JuliaMBD.allcomponents(b3))]
    

10-element Vector{Expr}:
 :(())
 :(var"##544" = (Float64)(t))
 quote
    var"##547" = var"##544"
end
 :(gainin = var"##547")
 :(var"##545" = K)
 :(gainout = var"##545" * gainin)
 quote
    var"##548" = gainout
end
 :(var"##546" = var"##548")
 :(y = (Float64)(var"##546"))
 quote
end

In [227]:
function Sys1()
    b = JuliaMBD.SubSystemBlock(:Sys1)
    K = JuliaMBD.ParameterPort(:K)
    b1 = JuliaMBD.Inport(:t, Float64)
    b2 = JuliaMBD.Gain()
    b3 = JuliaMBD.Outport(:y, Float64)
    JuliaMBD.add!(b, K)
    JuliaMBD.add!(b, b1)
    JuliaMBD.add!(b, b2)
    JuliaMBD.add!(b, b3)
    JuliaMBD.LineSignal(b1.env[:out], b2.env[:in], "s(t)")
    JuliaMBD.LineSignal(b2.env[:out], b3.env[:in], "s(t)")
    JuliaMBD.ConstSignal(:K, b2.env[:K])
    b.env[:K] = :K
    b
end


Sys1 (generic function with 1 method)

In [228]:
b = Sys1()

"Block(Sys1)"

In [229]:
expr = JuliaMBD.expr_function(b)

:(function Sys1_systemfunction(t::Float64; K::Any = K)
      ()
      var"##570" = (Float64)(t)
      begin
          var"##575" = var"##570"
      end
      var"##572" = var"##575"
      var"##571" = K
      var"##573" = var"##571" * var"##572"
      begin
          var"##576" = var"##573"
      end
      var"##574" = var"##576"
      y = (Float64)(var"##574")
      begin
      end
      (y = (Float64)(y),)
  end)

In [230]:
function Sys2()
    b = JuliaMBD.SubSystemBlock(:Sys2)
    b1 = JuliaMBD.Inport(:t, Float64)
    b2 = Sys1()
    b3 = JuliaMBD.Outport(:y, Float64)
    JuliaMBD.add!(b, b1)
    JuliaMBD.add!(b, b2)
    JuliaMBD.add!(b, b3)
    JuliaMBD.LineSignal(b1.env[:out], b2.env[:t], "s(t)")
    JuliaMBD.LineSignal(b2.env[:y], b3.env[:in], "s(t)")
    b
end


Sys2 (generic function with 1 method)

In [231]:
bb = Sys2()

"Block(Sys2)"

In [232]:
JuliaMBD.expr_function(bb)

:(function Sys2_systemfunction(t::Float64; )
      ()
      var"##577" = (Float64)(t)
      begin
          var"##586" = var"##577"
      end
      var"##579" = K
      t = (Float64)(var"##586")
      var"##578" = (Float64)(t)
      begin
          var"##583" = var"##578"
      end
      var"##580" = var"##583"
      var"##581" = var"##579" * var"##580"
      begin
          var"##584" = var"##581"
      end
      var"##582" = var"##584"
      y = (Float64)(var"##582")
      begin
          var"##587" = (Float64)(y)
      end
      var"##585" = var"##587"
      y = (Float64)(var"##585")
      begin
      end
      (y = (Float64)(y),)
  end)

In [236]:
expr = [JuliaMBD._expr(m) for m = JuliaMBD.tsort(JuliaMBD.allcomponents(p))]


MethodError: MethodError: no method matching allcomponents(::Main.JuliaMBD.OutPort{Any})
Closest candidates are:
  allcomponents(!Matched::Main.JuliaMBD.AbstractCompositeBlock) at /workspaces/JuliaMBD/src/_tsort.jl:26
  allcomponents(!Matched::Main.JuliaMBD.AbstractComponent) at /workspaces/JuliaMBD/src/_tsort.jl:37

In [237]:
b = JuliaMBD.Integrator()
JuliaMBD.LineSignal(b.env[:out], b.env[:in], "s(t)")

"Line(##598)"

In [238]:
JuliaMBD.tsort(JuliaMBD.allcomponents(b.env[:in]))

11-element Vector{Main.JuliaMBD.AbstractComponent}:
 "InPort(##594)"
 "Block(Outport)"
 "Parameter(initialcondition)"
 "OutPort(out)"
 "Parameter(saturationlimits)"
 "InPort(in)"
 "Block(Inport)"
 "OutPort(##593)"
 "InPort(##595)"
 "Block(IntegratorInner)"
 "OutPort(##596)"

In [239]:
expr = [JuliaMBD._expr(m) for m = JuliaMBD.tsort(JuliaMBD.allcomponents(b.env[:in]))]


11-element Vector{Expr}:
 :(())
 :(out = var"##594")
 :(())
 quote
    var"##598" = out
end
 :(())
 :(in = var"##598")
 :(var"##593" = in)
 quote
    var"##597" = var"##593"
end
 :(var"##595" = var"##597")
 :(var"##596" = var"##595")
 quote
end

In [240]:
b = JuliaMBD.Add(signs=[:+, :-])

"Block(Add)"

In [241]:
b.inports

2-element Vector{Main.JuliaMBD.AbstractInPortBlock}:
 "InPort(##600)"
 "InPort(##601)"